# Cyborg Cantina

__Retrieval-based__ chatbots are commonly used in customer service environments. User questions are constrained to the specific service being provided, which approximates a closed-domain conversation. However, people are known to be especially picky with their food. So how about creating a chatbot system that effectively answers all of a diner’s questions about the food on a restaurant menu?

In this project we will build a retrieval-based chatbot system for a restaurant serving Mexican cuisine. We’ll use a combination of __tf-idf__ scoring, __word embedding__ models, and a set of use-defined functions in order to answer any number of questions from a restaurant diner.

By the end of this project, we’ll have constructed a full retrieval-based chatbot system, conversed with our own chatbot creation, and improved the system by adding additional candidate responses!

## Chabot Input/Output

Let’s begin at the end! We must provide a way for our user to end a conversation once they have had all their questions answered. For that purpose, we have `exit_commands`, a list of strings commonly used as exit commands from a chatbot system.

Now we need to define a `.make_exit()` method with `self` and `user_message` as parameters.\
Within `.make_exit()`, we write a for loop over each object in `exit_commands`, check if the object also occurs in `user_message`.\
If the object does occur in `user_message`, we print a goodbye message to the console, and return `True`.\
If the object does not occur in `user_message`, our function does nothing.

In [1]:
from collections import Counter
from responses import responses, blank_spot
from user_functions import preprocess, compare_overlap, pos_tag, extract_nouns, compute_similarity
import spacy
#import en_core_web_md
word2vec = spacy.load("en_core_web_md")

exit_commands = ("quit", "goodbye", "exit", "no")

class ChatBot:
  
  #define .make_exit()
  def make_exit(self, user_message):
    for command in exit_commands:
      if command in user_message:
        print("Ok, have a good day!")
        return True
    

Of course, we should provide a method that allows our chatbot to chat:

In [2]:
class ChatBot:
    
    #define .make_exit()
    def make_exit(self, user_message):
        for command in exit_commands:
            if command in user_message:
                print("Ok, have a good day!")
                return True
    
    #define .chat()
    def chat(self):
        # write a welcoming prompt for a user question
        user_message = input("Hello there! How can I help you? ")
        while not self.make_exit(user_message):
            user_message = self.respond(user_message)
            

## Intent Classification

Let’s build a set of BoW models from our data.

In [3]:
class ChatBot:
    
    #define .make_exit()
    def make_exit(self, user_message):
        for command in exit_commands:
            if command in user_message:
                print("Ok, have a good day!")
                return True
    
    #define .chat()
    def chat(self):
        # write a welcoming prompt for a user question
        user_message = input("Hello there! How can I help you? ")
        while not self.make_exit(user_message):
            user_message = self.respond(user_message)


    #define .find_intent_match()
    def find_intent_match(self, responses, user_message):
        # create a Bag-of-Words model
        bow_user_message = Counter(preprocess(user_message))
        processed_responses = [Counter(preprocess(response)) for response in responses]
        # select the response that best matches the intent of the user message
        similarity_list = [compare_overlap(item, bow_user_message) for item in processed_responses]
        # select the index of the highest similarity score
        response_index = similarity_list.index(max(similarity_list))
        return responses[response_index]
    

Let’s test our `.find_intent_match()` method.

In [4]:
class ChatBot:
    
    #define .make_exit()
    def make_exit(self, user_message):
        for command in exit_commands:
            if command in user_message:
                print("Ok, have a good day!")
                return True
    
    #define .chat()
    def chat(self):
        # write a welcoming prompt for a user question
        user_message = input("Hello there! How can I help you? ")
        while not self.make_exit(user_message):
            user_message = self.respond(user_message)


    #define .find_intent_match()
    def find_intent_match(self, responses, user_message):
        # create a Bag-of-Words model
        bow_user_message = Counter(preprocess(user_message))
        processed_responses = [Counter(preprocess(response)) for response in responses]
        # select the response that best matches the intent of the user message
        similarity_list = [compare_overlap(item, bow_user_message) for item in processed_responses]
        # select the index of the highest similarity score
        response_index = similarity_list.index(max(similarity_list))
        return responses[response_index]
    
    
    # define .respond()
    def respond(self, user_message):
        best_response = self.find_intent_match(responses, user_message)
        entity = self.find_entities(user_message)
        print(best_response.format(entity))
        #print(best_response)
        input_message = input("Would you like something else? ")
        return input_message
        

## Entity Recognition and Response Selection

Let’s extract candidate entities from the user message.

In [5]:
class ChatBot:
    
    #define .make_exit()
    def make_exit(self, user_message):
        for command in exit_commands:
            if command in user_message:
                print("Ok, have a good day!")
                return True
            
            
    #define .chat()
    def chat(self):
        # write a welcoming prompt for a user question
        user_message = input("Hello there! How can I help you? ")
        while not self.make_exit(user_message):
            user_message = self.respond(user_message)


    #define .find_intent_match()
    def find_intent_match(self, responses, user_message):
        # create a Bag-of-Words model
        bow_user_message = Counter(preprocess(user_message))
        processed_responses = [Counter(preprocess(response)) for response in responses]
        # select the response that best matches the intent of the user message
        similarity_list = [compare_overlap(item, bow_user_message) for item in processed_responses]
        # select the index of the highest similarity score
        response_index = similarity_list.index(max(similarity_list))
        return responses[response_index]
    
    
    #define .find_entities()
    def find_entities(self, user_message):
        tagged_user_message = pos_tag(preprocess(user_message))
        message_nouns = extract_nouns(tagged_user_message)
        # fit a word2vec model on our candidate entities
        tokens = word2vec(" ".join(message_nouns))
        category = word2vec(blank_spot)
        word2vec_result = compute_similarity(tokens, category)
        # select the entity with the highest similarity score
        word2vec_result.sort(key=lambda x: x[2])
        if len(word2vec_result) > 0:
            return word2vec_result[-1][0]
        else:
            return blank_spot

    
    # define .respond()
    def respond(self, user_message):
        best_response = self.find_intent_match(responses, user_message)
        entity = self.find_entities(user_message)
        print(best_response.format(entity))
        #print(best_response)
        input_message = input("Would you like something else? ")
        return input_message

Let's test it out!

In [ ]:
#initialize ChatBot instance below:
chatbot_one = ChatBot()

#call .chat() method below:
chatbot_one.chat()